In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4777,application_1587988164357_1412,pyspark,idle,Link,Link,
4793,application_1587988164357_1428,pyspark,idle,Link,Link,
4794,application_1587988164357_1429,pyspark,idle,Link,Link,
4795,application_1587988164357_1430,pyspark,idle,Link,Link,
4797,application_1587988164357_1432,pyspark,idle,Link,Link,
4798,application_1587988164357_1433,pyspark,idle,Link,Link,
4800,application_1587988164357_1435,pyspark,idle,Link,Link,
4801,application_1587988164357_1436,pyspark,idle,Link,Link,
4802,application_1587988164357_1437,pyspark,idle,Link,Link,
4804,application_1587988164357_1439,pyspark,busy,Link,Link,


In [2]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4813,application_1587988164357_1448,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Loading data, these are snapshots of the all available data
# Calendar and trips are useful to filter the other dataframe according to the day
stop_times = spark.read.format('orc').load('/data/sbb/timetables/orc/stop_times/000000_0')
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')
trips = spark.read.format('orc').load('/data/sbb/timetables/orc/trips/000000_0')
calendar = spark.read.format('orc').load('/data/sbb/timetables/orc/calendar/000000_0')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
stop_times.show(3)
stops.show(3)
trips.show(3)
calendar.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:20:00|      04:20:00|8500010:0:3|            1|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:24:00|      04:24:00|8500020:0:3|            2|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021:0:5|            3|          0|            0|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
only showing top 3 rows

+-------+------------+----------------+----------------+-------------+--------------+
|stop_id|   stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+------------+----------------+----------------+-------------+------------

In [5]:
days_dict = {0: 'monday', 1: 'tuesday', 2: 'wednesday', 3: 'thursday', 4: 'friday'}

def day_trips(*day_ids):
    days = [days_dict[day_id] for day_id in day_ids]
    where_clause = " and ".join(days)

    day_services = calendar.where(where_clause).select('service_id')
    return day_services.join(trips, on='service_id').select('trip_id')

# Example for stop_times filtered on wednesday
stop_times_wed = day_trips(2).join(stop_times, on='trip_id')
stop_times_wed.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+------------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+------------+-------------+-----------+-------------+
|1.TA.11-12-B-j19-...|    08:30:00|      08:30:00|     8582915|            1|          0|            0|
|1.TA.11-12-B-j19-...|    08:31:00|      08:31:00|     8582916|            2|          0|            0|
|1.TA.11-12-B-j19-...|    08:33:00|      08:33:00|     8582917|            3|          0|            0|
|1.TA.11-12-B-j19-...|    08:35:00|      08:35:00|     8574680|            4|          0|            0|
|1.TA.12-1-A-j19-1...|    01:30:00|      01:30:00|8505000:0:11|            1|          0|            0|
+--------------------+------------+--------------+------------+-------------+-----------+-------------+
only showing top 5 rows

In [6]:
# Can't run, the count makes it timeout. I asked Tao why
#print('Full stop times have', stop_times.count(), 'entries, filtered has', stop_times_wed.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
zurich_pos = stops.where(column('stop_id')=='8503000').select('stop_lat', 'stop_lon').collect()
zurich_pos = (zurich_pos[0][0], zurich_pos[0][1])

def zurich_distance(x, y):
    return geo_distance(zurich_pos, (x,y)).km

stops_distance = stops.rdd.map(lambda x: (x['stop_id'], zurich_distance(x['stop_lat'], x['stop_lon'])))
stops_distance = spark.createDataFrame(stops_distance.map(lambda r: Row(stop_id=r[0], zurich_distance=r[1])))
stops_distance = stops_distance.filter(column('zurich_distance') <= 15)
stops_zurich = stops_distance.join(stops, on='stop_id')
print('Number of stations around Zurich:', stops_zurich.count())
stops_zurich.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Number of stations around Zurich:', 1880)
+-----------+------------------+--------------------+----------------+----------------+-------------+--------------+
|    stop_id|   zurich_distance|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-----------+------------------+--------------------+----------------+----------------+-------------+--------------+
|    8500926|11.510766966884365|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|             |              |
|    8502186|10.798985488832079|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|             |      8502186P|
|8502186:0:1|10.800041577194426|Dietikon Stoffelbach|47.3934666445388|8.39894248049007|             |      8502186P|
|8502186:0:2|10.801101793198619|Dietikon Stoffelbach|47.3935274568464|8.39894248049007|             |      8502186P|
|   8502186P|10.798985488832079|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|            1|              |
|    8502187|12.4092

In [8]:
for r in stops_zurich.collect():
    if r['stop_id'] == '8503000':
        print((r['stop_id'], {'name': r['stop_name'].encode('utf-8'), 'lat': r['stop_lat'], 'lon': r['stop_lon']}))
        break

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(u'8503000', {'lat': 47.3781762039461, 'lon': 8.54019357578468, 'name': 'Z\xc3\xbcrich HB'})

In [9]:
graph = nx.MultiDiGraph()
nodes = stops_zurich.rdd.map(lambda r: (r[0], {'name': r['stop_name'],
                                              'lat': r['stop_lat'],
                                              'lon': r['stop_lon']})).collect()
graph.add_nodes_from(nodes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
@udf
def convertToMinute(s):
    h, m, _ = s.split(':')
    h,m = int(h), int(m)
    
    return h*60+m

# Keep only travels around zurich
nodes_list = list(graph.nodes())
stop_times_zurich = stop_times.filter(column('stop_id').isin(nodes_list))
stop_times_zurich = stop_times_zurich.filter(column('stop_id').isin(nodes_list))
print('Number of stop times around zurich:', stop_times_zurich.count())
# Convert time information to minutes elapsed since 0am
stop_times_zurich = stop_times_zurich.withColumn('arrival_time', convertToMinute(column('arrival_time')))
stop_times_zurich = stop_times_zurich.withColumn('departure_time', convertToMinute(column('departure_time')))
# Add next stop to dataframe
stop_times_zurich_2 = (stop_times_zurich.withColumn('stop_sequence_prev', column('stop_sequence')-1)
                       .select('trip_id',
                               column('stop_id').alias('next_stop'),
                               column('stop_sequence_prev').alias('stop_sequence'),
                               column('arrival_time').alias('next_arrival_time')))
# Add trip duration
stop_times_zurich = stop_times_zurich.join(stop_times_zurich_2, on=['trip_id', 'stop_sequence']).orderBy('trip_id', 'stop_sequence')
stop_times_zurich = stop_times_zurich.withColumn('trip_duration', column('next_arrival_time')-column('departure_time'))
stop_times_zurich = stop_times_zurich.select('stop_id', 'arrival_time', 'departure_time', 'next_stop', 'trip_duration')
stop_times_zurich.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o7826.showString.
: java.util.concurrent.TimeoutException: Futures timed out after [300 seconds]
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:219)
	at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:223)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:201)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:136)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:367)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.

In [ ]:
max_trip_duration = 2 #duration in hour 
def filter_edge_on_time(edges_df, day_id, hour, minute):
    edges_df = edges_df.join(day_trips(day_id), on='trip_id')
    arrival_minute = hour*60+minute
    min_dep_time = arrival_minute - 60*60*max_trip_duration
    edges_df = edges_df.filter((col('departure_minute') > min_dep_time) & 
                                            (col('arrival_minute') <= arrival_minute))

    return edges_df

# Example of graph construction: Wednesday arrival at 11:30:00
edges = (filter_edge_on_time(edges_df, 2, 11, 30)
         .rdd.map(lambda r: (r['stop_id'], r['next_stop'], {'duration': r['trip_duration'],
                                                          'time': r['departure_minute']})).collect())
print('Number of edges:', len(edges))